# Exploratory Data Analysis of Steam Reviews
As we've explored the data we have available in the Steam review dataset, we have increased our understanding of the available information and have updated our research questions accordingly:
 - Is there any correlation between receiving a game for free and the positivity of review?
 - How does word choice affect the number of comments on a review?
 - How does the length of review correlate to the score the Steam algorithm gave it and the number of upvotes?

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from scripts import project_functions

data = project_functions.load_and_process('..\data\data_raw\*.csv')

## Comparing reviewers who paid vs. received the game for free
Ryan

## Comparing word choice to number of review comments
Maysey

## Comparing review length to Steam algorithm score and upvotes
Jaden